<a href="https://colab.research.google.com/github/DJCordhose/ml-workshop/blob/master/notebooks/process/flask-server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serving using Flask

Links
* making flask server directly run on Colab: https://stackoverflow.com/questions/54465816/how-to-use-flask-in-google-colaboratory-python-notebook
* logging
  * https://flask.palletsprojects.com/en/1.1.x/logging/
  * https://docs.python.org/3/howto/logging-cookbook.html
* In a real production environment you would either use a "real" web server that directs to flask or use a hosted solution
  * https://flask.palletsprojects.com/en/1.1.x/deploying/
  * https://en.wikipedia.org/wiki/Web_Server_Gateway_Interface

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
!curl -O https://raw.githubusercontent.com/DJCordhose/ml-workshop/master/prod/prod_pipeline_pca_std_rf_2.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 64710  100 64710    0     0   305k      0 --:--:-- --:--:-- --:--:--  303k


In [0]:
!tar -xzf prod_pipeline_pca_std_rf_2.tgz

In [2]:
!ls -lR

.:
total 128
drwxr-xr-x 2 root root  4096 Jul 25 11:11 data
-rw-r--r-- 1 root root 26688 Jul 25 10:43 export.ipynb
-rw-r--r-- 1 root root 30221 Jul 25 11:11 export-pipeline.ipynb
drwxr-xr-x 2 root root  4096 Jul 25 11:11 models
-rw-r--r-- 1 root root 24380 Jul 25 11:12 prod_pipeline_pca_std_rf_2.tgz
-rw-r--r-- 1 root root 26565 Jul 25 10:43 prod.tgz
drwxr-xr-x 1 root root  4096 Jul 19 16:14 sample_data
drwxr-xr-x 2 root root  4096 Jul 25 11:11 stats

./data:
total 28
-rw-r--r-- 1 root root 26783 Jul 25 11:11 insurance-customers-1500.csv

./models:
total 56
-rw-r--r-- 1 root root 56419 Jul 25 11:11 model.pickle

./sample_data:
total 55504
-r-xr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Jul 19 16:14 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jul 19 16:14 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Jul 19 16:14 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jul 19 16:14 mnist_train_small.csv
-r-xr-xr-x 1 root roo

In [0]:
import pickle

In [0]:
model_versions = pickle.load(open('stats/versions.pickle', 'rb'))

In [0]:
import numpy, sklearn, pandas

versions = {
    'numpy': numpy.__version__,
    'sklearn': sklearn.__version__, 
    'pandas': pandas.__version__,
    'model': 2,
    'model_name': 'pipeline_pca_std_rf'
}

In [0]:
assert model_versions == versions

In [0]:
import logging

logging.basicConfig(filename='req.log')

data_logger = logging.getLogger('DataLogger')
data_logger.setLevel(logging.INFO)

file_handler = logging.FileHandler('data.log', )

data_logger.addHandler(file_handler)

In [0]:
# loading models only once on startup
model = pickle.load(open('models/model.pickle', 'rb'))

def predict(speed, age, miles):
    sample = [[speed, age, miles]]

    result = int(model.predict(sample)[0])
    prediction = model.predict_proba(sample)[0].tolist()
    
    return result, prediction

In [9]:
predict(100, 48, 10)

(1, [0.025061955771027718, 0.6844328432545325, 0.2905052009744397])

In [10]:
from flask import Flask, request, jsonify
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

app = Flask(__name__)

@app.route("/ping")
def ping():
    return "pong"

@app.route('/predict', methods=['GET', 'POST'])
def do_predict():
    speed = request.json['speed']
    age = request.json['age']
    miles = request.json['miles']

    predicted_category, probabilities = predict(speed, age, miles)

    response = {
        'category': predicted_category,
        'prediction': probabilities,
    }
    
    dataset = {
        'out': response,
        'in': {
            'speed': speed, 'age': age, 'miles': miles
        }
    }

    data_logger.info(dataset)

    return jsonify(response)
  
import threading
threading.Thread(target=app.run, kwargs={'host': '0.0.0.0', 'port': 80}).start()  

172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [11]:
import requests
r = requests.get("http://172.28.0.2/ping")
print(r.status_code)
print(r.encoding)
print(r.apparent_encoding)
print(r.text)

200
utf-8
ascii
pong


In [12]:
r = requests.post("http://172.28.0.2/predict", json={'speed': 100, 'age': 48, 'miles': 10})
r.status_code

200

In [13]:
response = r.json()
response

{'category': 1,
 'prediction': [0.025061955771027718, 0.6844328432545325, 0.29050520097443977]}

In [14]:
response['prediction']

[0.025061955771027718, 0.6844328432545325, 0.29050520097443977]

In [15]:
!cat data.log

{'out': {'category': 1, 'prediction': [0.025061955771027718, 0.6844328432545325, 0.29050520097443977]}, 'in': {'speed': 100, 'age': 48, 'miles': 10}}
